In [1]:
import torch
import torch.utils.data as tud
import torch.nn as nn
import pickle
from tqdm.notebook import tqdm
import datetime
import matplotlib.pyplot as plt
import importlib
import sys
import pandas as pd

In [2]:
!pwd

/home/jarobyte/guemes/icdar/bg


In [3]:
sys.path.append("/home/jarobyte/guemes/lib")

In [4]:
from pytorch_decoding.seq2seq import Transformer

In [5]:
scratch = "/home/jarobyte/scratch/guemes/icdar/bg/"

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [7]:
char2i = pickle.load(open(scratch + "data/char2i.pkl", "rb"))

In [8]:
i2char = pickle.load(open(scratch + "data/i2char.pkl", "rb"))

# data

In [9]:
# train_size = 1000000

# dev_size = 1000000

train_size = 1000

dev_size = 100

## train

In [10]:
train_source = torch.load(scratch + "data/train_source.pt")[:train_size].to(device)
train_source.shape

torch.Size([1000, 102])

In [11]:
train_target = torch.load(scratch + "data/train_target.pt")[:train_size].to(device)
train_target.shape

torch.Size([1000, 102])

In [12]:
dev_source = torch.load(scratch + "data/dev_source.pt")[:dev_size].to(device)
dev_source.shape

torch.Size([100, 102])

In [13]:
dev_target = torch.load(scratch + "data/dev_target.pt")[:dev_size].to(device)
dev_target.shape

torch.Size([100, 102])

# model

In [14]:
model = Transformer(char2i, 
                    i2char, 
                    max_sequence_length = 110,
                    embedding_dimension = 512,
                    feedforward_dimension = 2048,
                    attention_heads = 8,
                    encoder_layers = 4,
                    decoder_layers = 4)
model.to(device)

log = model.fit(train_source, 
                train_target, 
                dev_source, 
                dev_target, 
                epochs = 1, 
                progress_bar = 2, 
                learning_rate = 10**-4)

Model: Transformer
Tokens in the input vocabulary: 191
Tokens in the output vocabulary: 191
Max sequence length: 110
Embedding dimension: 512
Feedforward dimension: 2048
Encoder layers: 4
Decoder layers: 4
Attention heads: 8
Activation: relu
Dropout: 0.0
Trainable parameters: 29,777,599



  0%|          | 0/1 [00:00<?, ?it/s]

Training started
Epochs: 1
Learning rate: 0.0001
Weight decay: 0
Epoch | Train                 | Development           | Minutes
      | Loss     | Error Rate | Loss     | Error Rate |
---------------------------------------------------------------


  0%|          | 0/10 [00:00<?, ?it/s]

    1 |   3.9725 |     86.114 |   3.7072 |     83.297 |     0.0


In [15]:
idx, probs = model.predict(dev_source)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

In [16]:
model.tensor2text(idx)

['<START>                                                                                                      ',
 '<START>                                                                                                      ',
 '<START>                                                                                                      ',
 '<START>                                                                                                      ',
 '<START>                                                                                                      ',
 '<START>                                                                                                      ',
 '<START>                                                                                                      ',
 '<START>                                                                                                      ',
 '<START>                                                                               

In [17]:
model.tensor2text(train_target[:10])

['<START>   91     Какво ли има  акъ? Когато тичахъ прѣзъ търга, извика ми единъ ковачъ, който току що б@ѣше <END>',
 '<START>  91     Какво ли има  акъ? Когато тичахъ прѣзъ търга, извика ми единъ ковачъ, който току що б@ѣше п<END>',
 '<START> 91     Какво ли има  акъ? Когато тичахъ прѣзъ търга, извика ми единъ ковачъ, който току що б@ѣше пр<END>',
 '<START>91     Какво ли има  акъ? Когато тичахъ прѣзъ търга, извика ми единъ ковачъ, който току що б@ѣше про<END>',
 '<START>1     Какво ли има  акъ? Когато тичахъ прѣзъ търга, извика ми единъ ковачъ, който току що б@ѣше проч<END>',
 '<START>     Какво ли има  акъ? Когато тичахъ прѣзъ търга, извика ми единъ ковачъ, който току що б@ѣше проче<END>',
 '<START>    Какво ли има  акъ? Когато тичахъ прѣзъ търга, извика ми единъ ковачъ, който току що б@ѣше прочел<END>',
 '<START>   Какво ли има  акъ? Когато тичахъ прѣзъ търга, извика ми единъ ковачъ, който току що б@ѣше прочелъ<END>',
 '<START>  Какво ли има  акъ? Когато тичахъ прѣзъ търга, извика 

In [18]:
model.tensor2text(train_source[:10])

["<START>— 91 — Какво ли има пакъ? Когато тичахъ нрФзъ търга, извика ми едииъ ковачъ, който току що б'Ьше <END>",
 "<START>— 91 — Какво ли има пакъ? Когато тичахъ нрФзъ търга, извика ми едииъ ковачъ, който току що б'Ьше н<END>",
 "<START> 91 — Какво ли има пакъ? Когато тичахъ нрФзъ търга, извика ми едииъ ковачъ, който току що б'Ьше нр<END>",
 "<START>91 — Какво ли има пакъ? Когато тичахъ нрФзъ търга, извика ми едииъ ковачъ, който току що б'Ьше нро<END>",
 "<START>1 — Какво ли има пакъ? Когато тичахъ нрФзъ търга, извика ми едииъ ковачъ, който току що б'Ьше нроч<END>",
 "<START> — Какво ли има пакъ? Когато тичахъ нрФзъ търга, извика ми едииъ ковачъ, който току що б'Ьше нроче<END>",
 "<START> — Какво ли има пакъ? Когато тичахъ нрФзъ търга, извика ми едииъ ковачъ, който току що б'Ьше нрочел<END>",
 "<START> — Какво ли има пакъ? Когато тичахъ нрФзъ търга, извика ми едииъ ковачъ, който току що б'Ьше нрочелъ<END>",
 "<START>— Какво ли има пакъ? Когато тичахъ нрФзъ търга, извика ми едииъ ковач